# Basic Monochrome MWIR Imager on a Drone


Computing the basic parameters of a monochrome imager on a drone. Valid for a single band.

## Loading the Imager Parameters

We have to start with the opticks package import (as the notebook sometimes fails to find it).

An imager is made up of three parts: Optics, Detector and (optionally) Read-out/Write Electronics. We load the configuration files for each part and initialise the `Imager` object.

In [37]:
# If opticks import fails, try to locate the module
import os

try:
    import opticks
except ModuleNotFoundError:
    os.chdir(os.path.join("..", ".."))
    os.getcwd()

In [38]:
from pathlib import Path

from opticks.imager_model.detector import Channel
from opticks.imager_model.imager import Imager

print(f"current working directory: {Path.cwd()}")

file_directory = Path("docs", "examples", "sample_drone_imager")
optics_file_path = file_directory.joinpath("optics.yaml")
detector_file_path = file_directory.joinpath("detector.yaml")
rw_electronics_file_path = file_directory.joinpath("rw_electronics.yaml")

# check whether input files exist
print(
    f"optics file:  [{optics_file_path}] (file exists:  {optics_file_path.is_file()})"
)
print(
    f"detector file exists:  [{detector_file_path}] (file exists:  {detector_file_path.is_file()})"
)
print(
    f"RW electronics file exists:  [{rw_electronics_file_path}] (file exists:  {rw_electronics_file_path.is_file()})"
)

# Init imager object
imager = Imager.from_yaml_file(
    optics_file_path, detector_file_path, rw_electronics_file_path
)

# shorthands
optics = imager.optics
detector = imager.detector
rw_electronics = imager.rw_electronics

# select the first channel
channel: Channel = next(iter(detector.params.channels.all.values()))

# binning status
binning_on = False if channel.binning == 1 else True

current working directory: /home/egemen/Projects/opticks
optics file:  [docs/examples/sample_drone_imager/optics.yaml] (file exists:  True)
detector file exists:  [docs/examples/sample_drone_imager/detector.yaml] (file exists:  True)
RW electronics file exists:  [docs/examples/sample_drone_imager/rw_electronics.yaml] (file exists:  True)


## Setting the Scene

Reference parameters for the camera position and motion with respect to the target are given below:

In [39]:
import numpy as np

from opticks import u

# positional params
# ---------------------
distance = 10.0 * u.km

print(f"target distance : {distance:~P} ")

target distance : 10.0 km 


Certain parameters vary with the light characteristics and therefore wavelength dependent. For a MWIR sensor, 3000 to 5000 can be selected. 

Note the widely used bands:

- blue: 450-485 nm
- green: 500-565 nm
- red: 625-750 nm
- nir: 750-1400 nm
- swir: 1400-3000 nm
- mwir: 3000-8000 nm

In [40]:
ref_wavelength = 4000 * u.nm

## Extracting the Imager Parameters

### Optical Parameters

Basic and derived optical parameters are given below for the optics:


In [41]:
print("Basic Optical Params:")

print(f"optics id: {optics.params.name}")
print(f"focal length : {optics.params.focal_length:~}")
print(f"aperture diameter : {optics.params.aperture_diameter:~}")
print(f"image diameter on focal plane : {optics.params.image_diam_on_focal_plane:~}")

print()
print("Derived Optical Params:")

print(f"f-number : {optics.f_number:.4}")
print(f"full optical fov : {optics.full_optical_fov:~P.4}")
print(f"aperture area : {optics.aperture_area.to("cm**2"):~P.6}")
print(f"spatial cut-off freq  : {optics.spatial_cutoff_freq(ref_wavelength):~P.5} (at {ref_wavelength:~P})")


Basic Optical Params:
optics id: Sample Reflective Optics
focal length : 300 mm
aperture diameter : 130 mm
image diameter on focal plane : 12.3 mm

Derived Optical Params:
f-number : 2.308
full optical fov : 2.349 deg
aperture area : 132.732 cm²
spatial cut-off freq  : 108.33 cy/mm (at 4000 nm)


### Detector Parameters

Basic and derived detector parameters are given below for the optics:

In [42]:
print("Basic Detector Params:")

print(f"detector id: {detector.params.name}")
print(f"detector type : {detector.params.detector_type}")
print(f"horizontal x vertical pixels (detector) : {detector.params.horizontal_pixels} x {detector.params.vertical_pixels}")
print(f"horizontal x vertical pixels (channel) : {channel.horizontal_pixels} x {channel.vertical_pixels}")
print(f"binning : {channel.binning if binning_on else 'None'}")
print(f"pixel pitch : {channel.pixel_pitch(False):~P} {f"({channel.pixel_pitch(True):~P} binned)" if binning_on else ''}")
print(f"TDI stages : {'None' if channel.tdi_stages == 1 else channel.tdi_stages}")

print()
print("Derived Detector Params:")

print(f"Nyquist freq : {channel.nyquist_freq(False):~P.4} {f"({channel.nyquist_freq(True):~P.4} binned)" if binning_on else ''}")
print(f"number of pixels (full frame) : {detector.pixel_count:~P.4}")
print(f"number of pixels (full frame, channel) : {channel.pixel_count_frame(False):~P.4} {f"({channel.pixel_count_frame(True):~P.4} binned)" if binning_on else ''}")


Basic Detector Params:
detector id: Sample Full-Frame Detector
detector type : full frame
horizontal x vertical pixels (detector) : 1280 x 720
horizontal x vertical pixels (channel) : 1280 x 720
binning : None
pixel pitch : 8 µm 
TDI stages : None

Derived Detector Params:
Nyquist freq : 62.5 cy/mm 
number of pixels (full frame) : 0.9216 Mpixel
number of pixels (full frame, channel) : 0.9216 Mpixel 


### Imager Geometry Parameters

The derived parameters are given below for the imager:

In [43]:
print(f"ifov : {channel.ifov(optics, False):~P.4} {f"({channel.ifov(optics, True):~P.4} binned)" if binning_on else ''}")
print(f"pixel solid angle : {channel.pix_solid_angle(optics, False):~P.4} {f"({channel.pix_solid_angle(optics, True):~P.4} binned)" if binning_on else ''}")
print(f"horizontal full fov: {channel.horizontal_fov(optics):~P.4}")
print(f"vertical full fov: {channel.vertical_fov(optics):~P.4}")

ifov : 1.528 mdeg 
pixel solid angle : 7.111×10⁻¹⁰ sr 
horizontal full fov: 1.956 deg
vertical full fov: 1.1 deg


### Geometric Projection Parameters

In [44]:
# Ground sample distance at distance
spatial_sample_distance_native = (
    (
        distance
        * (channel.pixel_pitch(False) * channel.binning / optics.params.focal_length)
    )
    .to_reduced_units()
    .to(u.m)
)
spatial_sample_distance = (
    (
        distance
        * (channel.pixel_pitch(True) * channel.binning / optics.params.focal_length)
    )
    .to_reduced_units()
    .to(u.m)
)

# image width and height assuming flat plate and constant Instantaneous FoV
image_width = (
    2 * np.tan(channel.ifov(optics, False) * channel.horizontal_pixels / 2.0) * distance
)

image_height = (
    2 * np.tan(channel.ifov(optics, False) * channel.vertical_pixels / 2.0) * distance
)

print(f"target distance : {distance:~P}")   
print(f"spatial sample distance : {spatial_sample_distance_native:~P.4} {f"({spatial_sample_distance:~P.4} binned)" if binning_on else ''} (at target distance)")
print(f"image width : {image_width:~.4P} (at target distance)")
print(f"image height : {image_height:~.4P} (at target distance)")

target distance : 10.0 km
spatial sample distance : 0.2667 m  (at target distance)
image width : 0.3414 km (at target distance)
image height : 0.192 km (at target distance)


### Timings

In [45]:
timings = detector.params.timings

print(f"frame rate :  {timings.frame_rate:~P.6} ({timings.frame_duration:~P.4})")
print(f"max integration duration : {timings.max_integration_duration:~P.4}")
print(f"actual integration duration : {timings.integration_duration:~P.4}")

frame rate :  29.97 Hz (33.37 ms)
max integration duration : 33.37 ms
actual integration duration : 13.34 ms


### Read-out/Write Electronics

In [46]:
print(f"pixel read rate : {channel.pix_read_rate(timings.frame_rate, False, False):~P.6} {f'({channel.pix_read_rate(timings.frame_rate, True, False):~P.6} binned)' if binning_on else ''} ({channel.name} channel)")

print(f"data write rate (uncompressed, incl. overheads) : {rw_electronics.data_write_rate(channel, detector, False, False):~P.6} {f'({rw_electronics.data_write_rate(channel, detector, True, False):~P.6} binned)' if binning_on else ''} ({channel.name} channel)")
print(f"data write rate (compressed, incl. overheads) : {rw_electronics.data_write_rate(channel, detector, False, True):~P.6} {f'({rw_electronics.data_write_rate(channel, detector, True, True):~P.6} binned)' if binning_on else ''} ({channel.name} channel)")

pixel read rate : 27.6204 Mpixel/s  (MWIR channel)
data write rate (uncompressed, incl. overheads) : 338.073 Mbit/s  (MWIR channel)
data write rate (compressed, incl. overheads) : 140.864 Mbit/s  (MWIR channel)
